In [2]:
# Imports
import cv2
import face_recognition
import numpy as np
from keras.models import model_from_json
from keras.models import load_model

# Load the emotion detection model
with open("machine_learning_model.json", "r") as json_file:
    loaded_model_json = json_file.read()
machine_learning_model = model_from_json(loaded_model_json)
machine_learning_model.load_weights("machine_learning_model.weights.h5")
print("Emotion detection model loaded.")

# Load the age detection model
age_model = load_model("age_detected_model.keras")
print("Age detection model loaded.")

# Emotion labels
emotions = {
    0: "Angry", 
    1: "Disgusted", 
    2: "Fearful", 
    3: "Happy", 
    4: "Neutral", 
    5: "Sad", 
    6: "Surprised"
}

# Function to get face encodings
def get_face_encoding(image_path):
    image = face_recognition.load_image_file(image_path)
    encodings = face_recognition.face_encodings(image)
    if len(encodings) > 0:
        return encodings[0]
    else:
        print(f"No face found in {image_path}")
        return None

# Load reference images and names
reference_face_encoding_1 = get_face_encoding("reference1.jpg")
reference_face_encoding_2 = get_face_encoding("reference2.jpg")
reference_face_encoding_3 = get_face_encoding("reference3.jpg")

known_face_encodings = []
known_face_names = []

if reference_face_encoding_1 is not None:
    known_face_encodings.append(reference_face_encoding_1)
    known_face_names.append("Minh Nguyen")
if reference_face_encoding_2 is not None:
    known_face_encodings.append(reference_face_encoding_2)
    known_face_names.append("Vishnu")
if reference_face_encoding_3 is not None:
    known_face_encodings.append(reference_face_encoding_3)
    known_face_names.append("Varun")

if len(known_face_encodings) == 0:
    print("No reference faces loaded. Exiting...")
    exit()

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()

    if ret:
        # Convert the frame from BGR to RGB for face recognition
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face locations and encodings
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Compare detected face encoding with known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "No Match!"

            if True in matches:
                match_index = matches.index(True)
                name = known_face_names[match_index]

            # Draw rectangle around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

            # Crop the face for emotion detection
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face_crop = gray_frame[top:bottom, left:right]
            try:
                face_crop_resized = cv2.resize(face_crop, (48, 48))
                face_crop_resized = np.expand_dims(np.expand_dims(face_crop_resized, -1), 0)

                # Predict emotion
                emotion_prediction = machine_learning_model.predict(face_crop_resized)
                max_index = int(np.argmax(emotion_prediction))
                emotion = emotions[max_index]
            except Exception as e:
                emotion = "Error"

            # Crop the face for age detection
            face_color = frame[top:bottom, left:right]
            try:
                face_color_resized = cv2.resize(face_color, (224, 224)) / 255.0
                face_color_resized = np.expand_dims(face_color_resized, axis=0)

                # Predict age
                age_prediction = age_model.predict(face_color_resized)
                predicted_age = int(age_prediction[0][0])
            except Exception as e:
                predicted_age = "N/A"

            # Display name, emotion, and age
            cv2.putText(frame, f"{name} ({emotion})", (left, top - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)  # White text
            cv2.putText(frame, f"Age: {predicted_age}", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)  # White text

        # Show the processed video feed
        cv2.imshow("Face Recognition, Emotion & Age Detection", frame)

    # Exit loop when 'q' is pressed
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Emotion detection model loaded.
Age detection model loaded.


2024-12-02 23:16:22.586 python[39376:2382232] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-12-02 23:16:26.569 python[39376:2382232] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-02 23:16:26.569 python[39376:2382232] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━